In [ ]:
from PIL import Image 
import numpy as np 
import os 
import time
import matplotlib.pyplot as plt  
%matplotlib inline

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models,transforms,datasets


In [ ]:
from utils_exams import *
import matplotlib.pyplot as plt 
%matplotlib inline 

def show_grid(tensor):
    tensor = tensor.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # code added here
    tensor = std*tensor + mean 

    # The next two lines are required to maintain the image values between 0 and 1
    tensor[tensor > 1] = 1 
    tensor[tensor < 0] = 0
    plt.figure(figsize = (20,2))    
    
    # code added here
    plt.imshow(tensor)

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
composed_transforms = transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),
                                          transforms.ToTensor(), normalize])

data_dir = 'data_exam'
dataset = {x: datasets.ImageFolder(os.path.join(data_dir, x), composed_transforms)
         for x in ['train', 'valid']}

In [ ]:
def to_shuffle(x):
    if x == 'train':
        return True
    else:
        return False

dataset_loader = {x: torch.utils.data.DataLoader(dataset[x], batch_size=4,
                                               shuffle=to_shuffle(x), num_workers=6)
                for x in ['train', 'valid']}

In [ ]:
inputs, classes = next(iter(dataset_loader['train']))
n_images = 4
out = torchvision.utils.make_grid(inputs[0:n_images])

show_grid(out)

In [ ]:
class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.categories = 1000 #Number of ImageNet categories
        #Insert your code here
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        self.block5 = nn.Sequential(
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        self.classify = nn.Sequential(
            nn.Linear(in_features = 512 * 7 * 7, out_features = 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(),
            nn.Linear(in_features = 4096, out_features = 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(),
            nn.Linear(in_features = 4096, out_features = self.categories),
            nn.Softmax()
        )

    def forward(self, x):
        #Insert your code here
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = x.view(x.size(0), -1) #Transformation before the classify layer.
        # Insert your code here
        x = self.classify(x)
        return x

In [ ]:
mynet = Network()

In [ ]:
from torch.utils import model_zoo
# Insert your code here
params_pre = model_zoo.load_url('https://download.pytorch.org/models/vgg16-397923af.pth')

In [ ]:
from utils_module_2 import *
# Insert your code here 
give_params_to_model(mynet, params_pre)

In [ ]:
for param in mynet.parameters():
    param.requires_grad = False

mynet.classify._modules['6'] = nn.Linear(in_features = 4096, out_features = 2)
mynet.classify._modules['7'] = nn.Softmax()

In [ ]:
learning_rate = 1e-4
optimizer_mynet = torch.optim.SGD(mynet.classify[6].parameters(),
                                  lr = learning_rate)

In [ ]:
def trainer_exam(model,dataloader, num_epochs,optimizer=None,train=True):

    r"""
    Implements a generic training function that can be used for training a network as well as for evaluating
    performance on a validation set

    Args:
        model: Network to be trained (or evaluated). Could be the complete network or a single layer of it
        dataloader: iterator over the dataset
        epochs (int, optional): number of epochs for training (default: 1)
        optimizer: Optimizer used for the training. Usually a torch.optim object
        train (bool, optional): Run the function in train or eval mode (default: True)
    """
    sizes = {'train': 257.0, 'valid': 140}
    if train:
        model.train()
        phase='train'
    else:
        model.eval()
        phase='valid'
    print("Phase is {}".format(phase))
    for epoch in range(num_epochs):
        total = 0
        running_loss = 0.0
        running_corrects = 0
        batch_counter = 1
        for inputs,classes in dataloader[phase]:
#             inputs , classes = Variable(torch.from_numpy(inputs)),Variable(torch.from_numpy(classes))
            outputs = model(inputs)
            loss_function = nn.CrossEntropyLoss()
            loss = loss_function(outputs,classes)
            optimizer = optimizer
            optimizer.zero_grad()
            if train:
                if optimizer is None:
                    raise ValueError('Pass optimizer for train mode')
#                 optimizer = optimizer
#                 optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            _,preds = torch.max(outputs.data,1)

            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)

            print("Cumulated loss of the " + str(batch_counter) + " first batches: {}".format(running_loss))
            batch_counter += 1
            #depending on py version
        epoch_loss = float(running_loss)
        epoch_acc = float(running_corrects)/sizes[phase]
        print('Loss: {:}, Acc: {:}'.format(epoch_loss, epoch_acc))

In [ ]:
trainer_exam(model=mynet,
             dataloader=dataset_loader, 
             num_epochs=10,
             optimizer=optimizer_mynet,
             train=True)

In [ ]:
trainer_exam(model=mynet,
             dataloader=dataset_loader, 
             num_epochs=1,
             optimizer=optimizer_mynet,
             train=False)